In [1]:
import os
import json
import re
import nltk
nltk.download('punkt_tab')
from nltk.tokenize import sent_tokenize, word_tokenize, PunktTokenizer
from gensim.models import Word2Vec

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/nazimaniltepe/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [2]:
json_data = []
for path, dirs, files in os.walk(f"./output"):
    for file in files:
        if file.endswith(".json"):
            f = open(os.path.join(path, file), encoding="utf-8-sig")
            json_data.append(json.load(f))
            f.close()
print("number of documents:", len(json_data))

number of documents: 215


In [3]:
text_data = [f"{' '.join([i['text'] for topic in j['topics'] for i in topic['items']])} {j['abstract']}" for j in json_data]
len(text_data)

215

In [4]:
sw_file = open('./stopwords.txt')
stop_words = sw_file.read()
stop_words = stop_words.split("\n")
sw_file.close()

In [5]:
regexes = [
    (r"([’’])", "'"),
    (r"([“”\"])", ""),
    (r"â", "a"),
    (r"î", "i"),
    (r"û", "u"),
    (r"Â", "A"),
    (r"Î", "İ"),
    (r"Û", "U"),
]

In [6]:
text_clean = text_data
for reg in regexes:
    text_clean = [re.sub(reg[0], reg[1], t) for t in text_clean]

text_lower = [t.replace("İ", "i").replace("I", "ı").lower() for t in text_clean]

# text_lower = [t for t in text_lower if t not in stop_words]

In [7]:
regexes2 = [
    (r"(,\s)", " "),
    (r"([\.\(])", ""), 
    (r"'", " "),
]
unique_words = []
text_test = text_lower
for reg in regexes2:
    text_test = [re.sub(reg[0], reg[1], t) for t in text_test]
for w in " ".join(text_test).split(" "):
    if w not in [ww[0] for ww in unique_words]:
        unique_words.append((w, 1))
    else:
        w_idx = [i for i, ww in enumerate(unique_words) if ww[0] == w][0]
        unique_words[w_idx] = (w, unique_words[w_idx][1] + 1)

In [14]:
unique_words_sorted = unique_words
unique_words_sorted.sort(reverse=True, key=lambda w: w[1])
unique_words_sorted = [w for w in unique_words_sorted if len(w[0]) < 5]

In [8]:
print(text_lower[0])

mayıs ayında tüketici fiyatları yüzde 0,15 oranında artarken yıllık enflasyon yüzde 6,51'e yükselmiştir. bu artışta enerji fiyatlarından kaynaklanan baz etkisi belirleyici olurken temel enflasyon göstergeleri ılımlı seyrini korumuştur. gıda ve alkolsüz içecekler grubunda yıllık enflasyon yüzde 7,07'ye yükselmiştir. işlenmiş gıda grubunda fiyatlar mayıs ayında yüzde 0,55 oranında artarken yıllık enflasyon yüzde 8,19'a gerilemiştir. grup enflasyonunda aylık bazda gözlenen artışın temel belirleyicisi ekmek ve tahıllar alt grubu olurken, diğer işlenmiş gıda fiyatlarının katkısı sınırlı kalmıştır. işlenmemiş gıda fiyatları ise mayıs ayında temelde taze sebze kaynaklı olarak yüzde 5,33 oranında gerilemiş, ancak baz etkisi nedeniyle grup yıllık enflasyonu artış kaydetmiştir. haziran ayına ilişkin öncü göstergeler bu grupta yıllık enflasyonunun belirgin olarak yükseleceğine işaret etmektedir. uluslararası petrol fiyatlarındaki düşüşün enerji enflasyonuna olumlu yansıması sürmüş, grup fiyatları

In [52]:
sent_detector = PunktTokenizer()
sentences = sent_detector.tokenize(" ".join(text_data))

In [91]:
sent_tokenize(" ".join(text_data), "turkish")
data = [word_tokenize(t, "turkish") for t in sent_tokenize(" ".join(text_lower))] 

In [92]:
model = Word2Vec(data, vector_size=100, window=10, min_count=1, workers=4)

In [93]:
len(model.wv.key_to_index)

12138

In [112]:
model.wv.similar_by_word("artış")

[('gerileme', 0.7931217551231384),
 ('azalış', 0.7836350798606873),
 ('yükseliş', 0.7253963947296143),
 ('artışa', 0.7001250982284546),
 ('sergileyen', 0.6701776385307312),
 ('yavaşlama', 0.666239857673645),
 ('bazda', 0.6590507626533508),
 ('gösteren', 0.6586082577705383),
 ('artışı', 0.6577882170677185),
 ('endeksindeki', 0.6574394702911377)]

In [113]:
model.wv.most_similar(positive=["artış", "üretim"], negative=["gerileme"])

[('verilerinin', 0.7286842465400696),
 ('firmaların', 0.704422116279602),
 ('yatırım', 0.698823094367981),
 ('imalât', 0.6820348501205444),
 ('firmalarının', 0.6810845136642456),
 ('değerlendirildiğinde', 0.6803269386291504),
 ('imalat', 0.6727310419082642),
 ('mallarına', 0.6698333621025085),
 ('satış', 0.6661445498466492),
 ('anket', 0.6617270112037659)]